# Datos iniciales

- $h_1=13$ m, altura del agua a la base del tanque.
- $h_2=59$, m, altura de la base del tanque al suelo.
- $P_1=101.325\times10^{3}$ Pa, presión atmosférica.
- $P_2=500\times10^{3}$ Pa, presión en el punto de interés.
- $D$ m, diámetro de la tubería.
- $\rho=997,087$ kg/m$^3$, densidad del agua a $25$ °C.
- $\mu=0.0008891$ Pa/s, viscosidad dinámica del agua a $25$ °C.
- $\varepsilon=0.0015\times10^{-3}$ m, rugosidad.
- $w$ J/m$^3$, trabajo.
- $L=183$ m, longitud $x$ de la tubería.
- $h_L$ J/m$^3$, pérdida por fricción.
- $g=9.80665$ m/s$^2$, gravedad.
    
![algoritmo](figuras/Diagrama.JPG)

# Planteamiento del modelo

Se tienen la ecuación general de Bernoulli:
    
    
$$P_1+\frac{\rho v_{1}^{2}}{2}+\rho g h_{1}+w=P_{2}+\frac{\rho v_{2}^{2}}{2}+\rho g h_{2}+h_{L}$$
    
La fórmula del número de Reynolds:

$$R_e=\frac{\rho v D}{\mu}$$

Fórmula de las pérdidas por fricción:

$$h_L=2f\frac{v^2L}{gD}$$

Para $f$ se utilizará la expresión:


$$f=2\left[\left(\frac{8}{R_e}\right)^{12}+\frac{1}{\left(A+B\right)^{3/2}}\right]^{1/12}$$

Donde:


- $$A=\left[2.457\ln{\left(\frac{1}{\left(\frac{7}{R_e}\right)^{0.9}+\frac{0.27\varepsilon}{D}}\right)}\right]^{16}$$
- $$B=\left(\frac{37.530}{R_e}\right)^{16}$$

# Solución

Como se conoce $Q$, se tiene que:

$$Q=vA\Rightarrow v=\frac{Q}{A}=\frac{4\cdot30\times10^{-3}\textrm{ m$^3$/s}}{\pi D^2}$$

$$v=38.197\times10^{-3}D^{-2}\textrm{ m/s}$$

Sustituyendo con la expresión de velocidad en la ecuación del número de Reynolds y la de fricción:

$$R_e=42.836\times10^{3}D^{-1}$$

De las expresiones de $f$, $A$ y $B$, se puede entender que $f=f(D)$, por lo que la expresión para $h_L$ en la ecuación de fricción queda como:

$$h_L=f(D)\frac{2.976\times10^{-4}L}{D^{5}}$$

Esta expresión se puede introducir dentro de la ecuación de Bernoulli, y tomando $w=0$ porque no hay presencia de ninguna bomba, además, tomando $h_{1}=0$ y el punto de referencia como la base de la tubería:

$$P_1=P_{2}+\frac{\rho v^{2}}{2}-\rho g h_{2}+f(D)\frac{2.976\times10^{-4}L}{D^{5}}$$

Reacomodando y sustituyendo con la velocidad y el valor de $\rho$:

$$f(D)\frac{2.976\times10^{-4}L}{D^{5}}=(P_1-P_2)+\rho gh_2-0.7274D^{-4}$$

Multiplicando por $D^5$ a ambos lados:

$$2.976\times10^{-4}Lf(D)=(P_1-P_2)D^5+\rho gh_2D^5-0.7274D$$
$$2.976\times10^{-4}Lf(D)=((P_1-P_2)+\rho gh_2)D^5-0.7274D$$

Sustituyendo con los valores conocidos:

$$178.232\times10^{3}D^5-0.7274D-0.05446f(D)=0$$

## Método numérico

Se plantea resolver la ecuación anterior por medio de Newton-Raphson, llamando $\Phi(D)=178.232\times10^{3}D^5-0.7274D-0.05446f(D)$, la solución tendría la forma:

$$D_{n+1}=D_{n}-\frac{\Phi(D_{n})}{\Phi'(D_{n})}$$

Donde:

$$\Phi'(D)=891.16\times^{3}D^{4}-0.7274-0.05446f'(D)$$

In [1]:
from numpy import log as ln

In [2]:
# parámetros del fluido como función del diámetro

def Re(D):
    return 42.836e3/D

def A(D):
    return (2.457*ln(1/((7/Re(D))**(0.9)+(0.27*0.0015e-3/D))))**16

def B(D):
    return (37.530/Re(D))**16

def f(D):
    return 2*((8/Re(D))**12+1/(A(D)+B(D))**(3/2))**(1/12)

In [3]:
# Función a la cual se le desean encontrar los ceros.
def Phi(D):
    return 178.232e3*D**5-0.7274*D-0.05446*f(D)

In [4]:
def Derivada(func,D):
    '''
    Función que calcula la derivada en un punto de una función determinada en un punto D.
    '''
    h=0.001*D
    delta=func(D+h)-func(D-h)
    return delta/(2*h)

In [5]:
# Derivada de Phi
def Phi1(D):
    return 891.16e3*D**4-0.7274-0.05446*Derivada(f,D)

In [6]:
# Newton-Raphson
n=0
Di=1
while n<1000:
    Di-=Phi(Di)/Phi1(Di)
    n+=1

In [7]:
Di

0.0450046477400378

In [8]:
Phi(Di)

8.348356728138384e-18

# Conclusión

- El diámetro mínimo obtenido tras dos iteraciones es de $4.5$ cm, evaluado en $Phi(D)$, se aproxima a $0$.